
# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

Run a local spark session to test your installation:

In [ ]:
!pip install pyspark==2.4.1

     |████████████████████████████████| 215.7MB 62kB/s 
     |████████████████████████████████| 204kB 18.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.1-py2.py3-none-any.whl size=216054607 sha256=599a3bf1cb6fe48cb70a03b68e4bf0f576c1419531105299c6d880e7e7427ce2
  Stored in directory: /root/.cache/pip/wheels/47/9b/57/7984bf19763749a13eece44c3174adb6ae4bc95b920375ff50
Successfully built pyspark


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Congrats! Your Colab is ready to run Pyspark. Let's build a simple Linear Regression model.

# Linear Regression Model


Linear Regression model is one the oldest and widely used machine learning approach which assumes a relationship between dependent and independent variables. For example, a modeler might want to predict the forecast of the rain based on the humidity ratio. Linear Regression consists of the best fitting line through the scattered points on the graph and the best fitting line is known as the regression line.

The goal of this exercise to predict the housing prices by the given features. Let's predict the prices of the Boston Housing dataset by considering MEDV as the output variable and all the other variables as input.

Download the dataset from [here](https://github.com/asifahmed90/pyspark-ML-in-Colab/blob/master/BostonHousing.csv) and keep it somewhere on your computer. Load the dataset into your Colab directory from your local system:

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

dataset = spark.read.csv('/content/sample_data/california_housing_train.csv',inferSchema=True, header =True)

In [ ]:
dataset.toPandas().head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


Notice that we used InferSchema inside read.csv mofule. InferSchema enables us to infer automatically different data types for each column.

Let us print look into the dataset to see the data types of each column:

In [ ]:
dataset.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



Next step is to convert all the features from different columns into a single column and let's call this new vector column as 'Attributes' in the outputCol.

In [ ]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 
                                       'population', 'households', 'median_income'], outputCol = 'Attributes')

output = assembler.transform(dataset)

#Input vs Output
finalized_data = output.select("Attributes","median_house_value")

finalized_data.show()

+--------------------+------------------+
|          Attributes|median_house_value|
+--------------------+------------------+
|[-114.31,34.19,15...|           66900.0|
|[-114.47,34.4,19....|           80100.0|
|[-114.56,33.69,17...|           85700.0|
|[-114.57,33.64,14...|           73400.0|
|[-114.57,33.57,20...|           65500.0|
|[-114.58,33.63,29...|           74000.0|
|[-114.58,33.61,25...|           82400.0|
|[-114.59,34.83,41...|           48500.0|
|[-114.59,33.61,34...|           58400.0|
|[-114.6,34.83,46....|           48100.0|
|[-114.6,33.62,16....|           86500.0|
|[-114.6,33.6,21.0...|           62000.0|
|[-114.61,34.84,48...|           48600.0|
|[-114.61,34.83,31...|           70400.0|
|[-114.63,32.76,15...|           45000.0|
|[-114.65,34.89,17...|           69100.0|
|[-114.65,33.6,28....|           94900.0|
|[-114.65,32.79,21...|           25000.0|
|[-114.66,32.74,17...|           44000.0|
|[-114.67,33.92,17...|           27500.0|
+--------------------+------------

Here, 'Attributes' are in the input features from all the columns and 'medv' is the target column.
Next, we should split the training and testing data according to our dataset (0.8 and 0.2 in this case).

In [ ]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'median_house_value')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

+--------------------+------------------+------------------+
|          Attributes|median_house_value|        prediction|
+--------------------+------------------+------------------+
|[-124.27,40.69,36...|           79000.0|156082.97486890526|
|[-124.26,40.58,52...|          111400.0|163790.80893250462|
|[-124.21,41.77,17...|           68400.0| 80688.09940900002|
|[-124.17,40.75,13...|          116100.0|200317.63970201882|
|[-124.17,40.76,26...|           82800.0|136887.13028532919|
|[-124.17,40.78,39...|           68300.0|109145.39687579265|
|[-124.17,40.79,43...|           70500.0|128383.63776316913|
|[-124.17,40.8,52....|           75500.0|127812.78410826018|
|[-124.16,40.78,46...|           81800.0|201873.36267728452|
|[-124.16,40.78,50...|           85400.0|158874.38886260847|
|[-124.16,40.79,52...|           76900.0|128709.90889399545|
|[-124.15,40.76,24...|          100600.0| 156419.3661029744|
|[-124.15,40.8,47....|           74100.0|121725.15246195346|
|[-124.15,40.81,50...|  

We can also print the coefficient and intercept of the regression model by using the following command:

In [ ]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-43668.0916, -43253.8886, 1139.1982, -8.034, 133.3104, -37.126, 22.5972, 40567.8307])
The Intercept of the model is : -3671402.124585


# Basic Statistical Analysis

Once we are done with the basic linear regression operation, we can go a bit further and analyze our model statistically by importing RegressionEvaluator module from Pyspark.

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)



RMSE: 69708.729
MSE: 4859306896.166
MAE: 50792.566
r2: 0.634
